## Neural Net Classification of Cars with (and without) Dimensionality Reduction (PCA)

Shout out to Meigarom for <a href="https://www.kdnuggets.com/2018/07/dimensionality-reduction-pca-improve-classification-results.html">this excellent blog post</a>

### (1) Ingest the data

In [17]:
# Load library
library( dplyr )

# Load dataset
data = read.csv( "VEHICLES.csv", stringsAsFactor = FALSE )

In [18]:
head(data)

COMPACTNESS,CIRCULARITY,DIST_CIRCULARITY,RADIUS_RATIO,PR_AXIS_ASPECT_RATIO,MAX_LENGTH_ASPECT_RATIO,SCATTER_RATIO,ELONGATEDNESS,PR_AXIS_RECTANGULARITY,MAX_LENGTH_RECTANGULARITY,SC_VAR_MAJOR,SC_VAR_MINOR,SC_RAD_GYRATION,SKEW_MIN,SKEW_MAJ,KURT_MIN,KURT_MAJ,HOLLOWS,RESPONSE
95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197,van
91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199,van
104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,saab
93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207,van
85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,bus
107,57,106,172,50,6,255,26,28,169,280,957,264,85,5,9,181,183,bus


### (2) Pick out Saabs and Busses from the dataset and binarize the response variable


In [19]:
#Transform Dataset
# 
names(data)[names(data) == 'RESPONSE'] <- 'class'

dataset = data %>% 
            filter( class == "bus" | class == "saab" ) %>%
            transform( class = ifelse( class == "saab", 0, 1 ) )

head(dataset)



COMPACTNESS,CIRCULARITY,DIST_CIRCULARITY,RADIUS_RATIO,PR_AXIS_ASPECT_RATIO,MAX_LENGTH_ASPECT_RATIO,SCATTER_RATIO,ELONGATEDNESS,PR_AXIS_RECTANGULARITY,MAX_LENGTH_RECTANGULARITY,SC_VAR_MAJOR,SC_VAR_MINOR,SC_RAD_GYRATION,SKEW_MIN,SKEW_MAJ,KURT_MIN,KURT_MAJ,HOLLOWS,class
104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,0
85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,1
107,57,106,172,50,6,255,26,28,169,280,957,264,85,5,9,181,183,1
97,43,73,173,65,6,153,42,19,143,176,361,172,66,13,1,200,204,1
93,44,98,197,62,11,183,36,22,146,202,505,152,64,4,14,195,204,0
90,34,66,136,55,6,123,54,17,118,148,224,118,65,5,26,196,202,0


In [20]:
#convert the dataset to numeric
dataset = as.data.frame( sapply( dataset, as.numeric ) )
head(dataset)


COMPACTNESS,CIRCULARITY,DIST_CIRCULARITY,RADIUS_RATIO,PR_AXIS_ASPECT_RATIO,MAX_LENGTH_ASPECT_RATIO,SCATTER_RATIO,ELONGATEDNESS,PR_AXIS_RECTANGULARITY,MAX_LENGTH_RECTANGULARITY,SC_VAR_MAJOR,SC_VAR_MINOR,SC_RAD_GYRATION,SKEW_MIN,SKEW_MAJ,KURT_MIN,KURT_MAJ,HOLLOWS,class
104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,0
85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,1
107,57,106,172,50,6,255,26,28,169,280,957,264,85,5,9,181,183,1
97,43,73,173,65,6,153,42,19,143,176,361,172,66,13,1,200,204,1
93,44,98,197,62,11,183,36,22,146,202,505,152,64,4,14,195,204,0
90,34,66,136,55,6,123,54,17,118,148,224,118,65,5,26,196,202,0


### (3) Split into train and test set

In [21]:
# Spliting training and testing dataset
#(1) Create index
index = sample( 1:nrow( dataset ), nrow( dataset ) * 0.6, replace = FALSE ) 

#(2) Use indec to create train and test set
trainset = dataset[ index, ]
test = dataset[ -index, ]

#(3) remove class column from 'testset'
testset = test %>% select( -class )



### (4) Build a neural net

In [22]:
# Building a neural network (NN)
library( neuralnet )
n = names( trainset )
f = as.formula( paste( "class ~", paste( n[!n %in% "class"], collapse = "+" ) ) )
nn = neuralnet( f, trainset, hidden = 4, linear.output = FALSE, threshold = 0.01 )

plot( nn, rep = "best" )

In [23]:
# Testing the result output
nn.results = compute( nn, testset )

results = data.frame( actual = test$class, prediction = round( nn.results$net.result ) )


In [24]:
tail(results)

,actual,prediction
406,0,1
410,1,1
421,0,1
423,1,1
433,1,1
434,0,1


### (5) Check out the results

In [25]:


# Confusion matrix
t = table( results )
print(t)
# print( confusionMatrix( t ) )

      prediction
actual  1
     0 88
     1 86


#### ... well, basically it predicted that everything was a bus ... so lets try using PCA and then rerun the model

### (6) Run the PCA on the train and test sets

In [26]:
# PCA
pca_trainset = trainset %>% select( -class )
pca_testset = testset
pca = prcomp( pca_trainset, scale = T )

# variance
pr_var = ( pca$sdev )^2 

# % of variance
prop_varex = pr_var / sum( pr_var )

# Plot
plot( prop_varex, xlab = "Principal Component", 
                  ylab = "Proportion of Variance Explained", type = "b" )

In [27]:
# Scree Plot
plot( cumsum( prop_varex ), xlab = "Principal Component", 
                            ylab = "Cumulative Proportion of Variance Explained", type = "b" )

### (7) Run the new dataset throught the neural net

In [28]:
# Creating a new dataset
train = data.frame( class = trainset$class, pca$x )
t = as.data.frame( predict( pca, newdata = pca_testset ) )

new_trainset = train[, 1:9]
new_testset =  t[, 1:8]

# Build the neural network (NN)
library( neuralnet )
n = names( new_trainset )
f = as.formula( paste( "class ~", paste( n[!n %in% "class" ], collapse = "+" ) ) )
nn = neuralnet( f, new_trainset, hidden = 4, linear.output = FALSE, threshold=0.01 )

# Plot the NN
plot( nn, rep = "best" )

In [29]:
# Test the resulting output
nn.results = compute( nn, new_testset )

# Results
results = data.frame( actual = test$class, 
                      prediction = round( nn.results$net.result ) )

# Confusion Matrix
# library( caret )
t = table( results ) 
print(t)
# print( confusionMatrix( t ) )

      prediction
actual  0  1
     0 83  5
     1  1 85


#### ... Much better results